# Peak Finding in xarray DataArrays

There are no existing libraries for the purpose, and reducing n-dimensional tensors down to 1D arrays for the scipy peak finding algorithm is subjective as the user has to define what order to unfold the tensor.

In [ ]:
%reload_ext autoreload
%autoreload 3 --print

import xarray as xr
from pca_analysis.get_sample_data import get_zhang_data
from pca_analysis import xr_signal

xr.set_options(display_expand_data=False)

raw_data: xr.DataArray = get_zhang_data()
raw_data.name = "raw_data"


This tensor has 3 modes. We want to detect every peak at every point. There are some libraries for tensor flow, but generally speaking I'm sure that a 1D approach is best. For example


In [ ]:
raw_data


<xarray.DataArray 'raw_data' (sample: 44, time: 31, mz: 200)> Size: 2MB
0.001538 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
Coordinates:
  * sample   (sample) <U8 1kB 'ARG-BNS1' 'ARG-DDA1' ... 'SOU-SPI1' 'SOU-SRE1'
  * time     (time) float64 248B 16.44 16.46 16.47 16.48 ... 16.79 16.81 16.82
  * mz       (mz) float64 2kB 5.0 6.0 7.0 8.0 9.0 ... 201.0 202.0 203.0 204.0

In [ ]:
# from pca_analysis import xr_signal
from pca_analysis.xr_signal import find_peaks_array

ds = raw_data.to_dataset()
test_ds = ds.sel(mz=46).isel(sample=slice(5, 10))

res = test_ds["raw_data"].pipe(find_peaks_array, grouper=["sample", "mz"])

test_ds = test_ds.merge(res)
test_ds


<xarray.Dataset> Size: 3kB
Dimensions:   (sample: 5, time: 31)
Coordinates:
  * sample    (sample) <U8 160B 'ARG-SAL1' 'AUS-CAV1' ... 'AUS-HAR1' 'AUS-IB41'
  * time      (time) float64 248B 16.44 16.46 16.47 16.48 ... 16.79 16.81 16.82
    mz        float64 8B 46.0
Data variables:
    raw_data  (sample, time) float64 1kB 0.0001484 0.0001469 ... 0.0003215
    peaks     (sample, time) float64 1kB nan nan 0.0001905 ... nan 0.0003849 nan

In [ ]:
fig = xr_signal.facet_plot_signal_peaks(
    ds=test_ds,
    line_key="raw_data",
    marker_key="peaks",
    x="time",
    y="au",
    grouper=["sample", "mz"],
    col_wrap=2,
)
fig.update_layout(height=1500).show()


As we can see, peak finding is successful. It would be useful to add the viz function as an optional output of the peak finding function - given an xarray input, find the peaks and return as a dataset, providing a viz if specified.

In [ ]:
ds, fig = xr_signal.find_peaks_dataset(
    ds=test_ds,
    array_key="raw_data",
    grouper=["sample", "mz"],
    return_viz=True,
    x="time",
    col_wrap=3,
)
fig


And as we can see the convenience function works as expected, producing the same result as above.

## Conclusion

We have established a foundation of peak finding on xarray DataArrays and Dataset objects, including result calculation and visualisation. We have also established a convenience function for working at the Dataset level
rather than DataArray.